In [1]:
# Import dependencies 
import pandas as pd
import json
import requests
import os
from dotenv import load_dotenv
import numpy as np

In [2]:
# Load the wine.csv into a DataFrame.
wine_reviews_df = pd.read_csv(r'C:\Users\fares\Final_project\project_3_nlp_wine_recommendation\Resources\nlp_3.1_final.csv')
# Display the first 20 headlines 
wine_reviews_df

,Name,Color,Alcohol_Percentage,Judge_Rating,Review_Notes,Wine_Bottle_Label,URL,Price,color,aromas,flavors,finish,mouthfeel,aromas_aroma,categories,subcategories,outer_ring_attributes
0,.Beb 2007,Rosé,13.5,15.5,Bright red color and initial aromas dominated ...,It has an attractive color and an intense arom...,https://osvinhos.blogspot.com/2010/03/1232-beb...,5.75,"['red', 'Bright', 'rosé']","['initial aromas', 'red color', 'Bright red co...","['fruity', 'pleasant presence', 'interesting s...","['average end', 'persistent flavor']",[],"['persistent flavor', 'fruity notes', 'red col...",[],[],[]
1,.Beb 2009,Red,14.0,16.0,"Reddish color and very aromatic nose, where th...",The careful choice of the best installments of...,https://osvinhos.blogspot.com/2013/08/2803-beb...,8.75,['red'],['aromatic nose'],"['fresh', 'delicate', 'beautiful dose', 'spice...","['median persistence', 'average length']","['seductive texture', 'round tannins']","['fresh set', 'seductive texture', 'subtle bar...",[],[],[]
2,.Beb 2010,Red,14.0,16.0,It presents a pleasantly concentrated ruby ​​c...,The careful choice of the best installments of...,https://osvinhos.blogspot.com/2014/01/2988-beb...,8.75,"['red', 'ruby']","['floral', 'fruity']","['fruity', 'attractive wine', 'several spices'...",['long length'],['round tannins'],"['Violet', 'delicate spicy notes', 'very fruit...",['Floral'],['Colored Flowers'],['Violet']
3,.Beb 2011,White,13.5,15.5,Bright yellow color and delicately marked nose...,We associated a very old wardrobe vineyard wit...,https://osvinhos.blogspot.com/2013/09/2820-beb...,8.75,"['Bright', 'white']",[],"['yellow color', 'medium structure', 'Bright y...",[],[],"['bright yellow color', 'floral touch', 'tropi...",[],[],[]
4,.Beb 2012,White,13.5,16.0,It has a slightly pale yellow color and a nose...,We associated the structure and complexity of ...,https://osvinhos.blogspot.com/2015/06/3639-beb...,5.75,['white'],"['pale yellow color', 'yellow color', 'fruity'...","['oaky', 'fresh', 'elegant mineral', 'mineral ...",[],['well acidity'],"['well acidity', 'suggestive vegetable', 'flor...",['Maturation in Oak Barrel'],['Spices'],['Vanilla']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6343,Zé da Leonor Reserva 2013,White,13.0,15.5,Yellow color and nose rich in floral and citru...,"Castes: Arinto, Gouveio and Viognier. Zé da Le...",https://osvinhos.blogspot.com/2015/11/3816-ze-...,5.75,['white'],"['fruity', 'Yellow color']","['fruity', 'mineral notes', 'fresh']",['medium length'],['smooth acidity'],"['smooth acidity', 'fresh wine', 'citrus aroma...",[],[],[]
6344,Zéfyro 2007,White,13.0,16.0,"It is a modern wine, where the nose is attract...",NaN,https://osvinhos.blogspot.com/2010/06/1336-zef...,8.75,['white'],[],"['floral', 'modern wine', 'very mineral', 'las...",[],['interesting acidity'],"['last complex', 'modern wine', 'very mineral'...",[],[],[]
6345,Zéfyro 2013,White,12.5,15.0,"Yellow color with delicate gold notes, in the ...","(...) Island, which in the bowels of the deep ...",https://osvinhos.blogspot.com/2017/01/4287-zef...,8.75,"['gold', 'white']",['fruity'],"['gold notes', 'vegetable touch', 'delicate', ...",['short length'],['correct acidity'],"['short length', 'slight vegetable touch', 'fl...",[],[],[]
6346,Zom Touriga Nacional 2011,Red,14.5,17.0,Ruby color avioleted and nose rich in floral a...,Zom is located in the heart of the Douro Inter...,https://osvinhos.blogspot.com/2017/06/4428-zom...,13.75,"['Ruby', 'red']",['forest aromas'],"['pleasant structure', 'brief barrel', 'delica...",[],"['fruity', 'polished tannins', 'interesting vo...","['black fruit', 'barrel notes', 'forest aromas...",[],[],[]


## Preprocessing 

In [3]:
wine_reviews_df.columns

Index(['Name', 'Color', 'Alcohol_Percentage', 'Judge_Rating', 'Review_Notes',
       'Wine_Bottle_Label', 'URL', 'Price', 'color', 'aromas', 'flavors',
       'finish', 'mouthfeel', 'aromas_aroma', 'categories', 'subcategories',
       'outer_ring_attributes'],
      dtype='object')

In [4]:
# Define function to categorize wines into price intervals
def categorize_price(min_price):
    if min_price < 10:
        return "Low ($5 - $10)"
    elif 10 <= min_price < 20:
        return "Affordable ($10 - $20)"
    elif 20 <= min_price < 50:
        return "Mid-Range ($20 - $50)"
    elif 50 <= min_price < 100:
        return "Premium ($50 - $100)"
    else:
        return "Luxury ($100+)"

# Apply the function to create a new column
wine_reviews_df["Price_Interval"] = wine_reviews_df["Price"].apply(categorize_price)

In [5]:
# Define function to categorize wines into alchohol content intervals
def categorize_alcohol_content(alcohol_content):
    if alcohol_content < 10:
        return "Low (0% - 10%)"
    elif 10 <= alcohol_content < 12:
        return "Moderate (10% - 12%)"
    elif 12 <= alcohol_content < 14:
        return "High (12% - 14%)"
    else:
        return "Very High (14%+)"
    
# Apply the function to create a new column
wine_reviews_df["Alcohol_Content"] = wine_reviews_df["Alcohol_Percentage"].apply(categorize_alcohol_content)

In [6]:
# Define flavor categories with associated keywords
flavor_map = {
    "Floral": ["violet","Floral", "iris", "rose", "flowers", "blossom", "perfume", "floral", "lavender","vanilla" "orange blossom", "rose petal", "hibiscus", "jasmine", "geranium", "lilac", "honeysuckle", "acacia", "magnolia", "dandelion", "chamomile", "elderflower", "marigold", "daisy", "tulip", "daffodil", "carnation", "pansy", "poppy", "lily", "orchid", "dahlia", "peony", "sunflower", "tuberose", "crocus", "clematis", "freesia", "lupine", "snapdragon", "zinnia", "aster", "cosmos", "daisy", "dianthus", "gerbera", "larkspur", "lupine", "narcissus", "pansy", "petunia", "phlox", "salvia", "snapdragon", "sunflower", "tulip", "violet", "zinnia"],
    "Fruity": ["peach", "pear", "apple", "berry", "fruit","jammy", "strawberry", "raspberry", "cherry"],
    "Citrus": ["lemon","Citrus", "lime", "grapefruit", "orange"],
    "Dark-fruit": ["blackberry", "black cherry", "plum", "dark fruit"],
    "Herbal": ['Herbal',"green pepper", "herbaceous", "mint", "grass"],
    "Buttery": ["Buttery","butter", "creamy", "vanilla", "toasty"],
    "Spicy": ["Spicy","spice", "pepper", "clove", "cinnamon"],
    "Earthy": ["Earthy","leather", "mushroom", "truffle", "earth"],
    "Smoky": ["smoky","smoke", "bacon", "charcoal", "ash", "tar"],
    "Nutty": ["nutty","nut", "almond", "hazelnut"],
    "Tropical": ["pineapple", "mango", "banana", "tropical", "coconut", "guava", "passion fruit", "lychee", "papaya", "kiwi", "melon"],
    "Sweet": ["honey", "sugar", "sweet", "caramel", "butterscotch", "toffee"],
    "Bold": ["intense", "bold", "rich fruit","coffee", "chocolate", "espresso", "licorice"],
    "Fresh": ["fresh", "clean", "crisp", "zesty", "refreshing"],
}


In [7]:
def classify_flavor(Flavor):
    if pd.isna(Flavor):  # Handle missing values
        return "Unknown"
    
    Flavor_lower = Flavor.lower()  # Convert to lowercase
    
    for category, keywords in flavor_map.items():
        if any(word in Flavor_lower for word in keywords):
            return category  # Return the first matching category
    
    return "Other"  # Default if no keywords match


In [8]:
wine_reviews_df["Flavors Class"] = wine_reviews_df["flavors"].apply(classify_flavor)


In [9]:
wine_reviews_df["Flavors Class"].value_counts() 

Flavors Class
Fruity        3569
Floral        1486
Other          867
Spicy          150
Bold           126
Fresh           82
Earthy          34
Buttery          9
Smoky            8
Sweet            7
Dark-fruit       5
Herbal           4
Tropical         1
Name: count, dtype: int64

In [10]:

wine_reviews_df['Color'].value_counts()

Color
Red      3526
White    2429
Rosé      390
TNTO        2
TEJO        1
Name: count, dtype: int64

In [11]:
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")


In [12]:
 #Define word categories
color_shades = {
    "red","ruby", "garnet", "crimson", "scarlet","pink", "burgundy", "mahogany", "rosé", "brick","purple", "bright red" , "reddish ","light red","deep red", "dark red","grenat"
    "white","golden", "yellow", "straw", "pale yellow", "amber", "bright citrine","citrine","honey","grenade", "bright yellow","bright gold", "light gold", "light yellow", "light straw", "light amber",
    "rosé","pink", "salmon", "peach", "coral", "copper", "blush","garnet", "crimson", "scarlet", "burgundy", "mahogany", "brick"}
sensations = {"smooth", "tannic", "silky", "crisp", "balanced", "refreshing", "harsh",
              "full-bodied", "elegant", "round", "soft", "lively", "delicate", "rich", "dry", "astringent","sweet", "juicy", "creamy", "herbal",'fresh', "full-bodied", "light-bodied", "medium-bodied",}

In [13]:
def extract_wine_details(text):
    """
    Extracts color shades, flavors, and sensations from wine reviews using NLP.
    """
    doc = nlp(text.lower())  # Convert to lowercase for matching
    extracted_colors,extracted_sensations = set(), set()

    for token in doc:
        word = token.text
        if word in color_shades:
            extracted_colors.add(word)
        if word in sensations:
            extracted_sensations.add(word)

    return (
        ", ".join(extracted_colors) if extracted_colors else "Unknown",
        ", ".join(extracted_sensations) if extracted_sensations else "Unknown"
    )


In [14]:
# Extract color, flavor, and sensation
wine_reviews_df[['Color_shades', 'Sensation']] = wine_reviews_df['Review_Notes'].apply(lambda x: pd.Series(extract_wine_details(x)))



In [15]:
wine_reviews_df['Color_shades'].value_counts()

Color_shades
Unknown                 1313
yellow                  1000
ruby                     859
red, ruby                842
red                      571
                        ... 
golden, honey, amber       1
pink, ruby                 1
golden, straw              1
amber                      1
golden, peach, honey       1
Name: count, Length: 64, dtype: int64

In [16]:

# replace the unknown values in color shades with the Color values
wine_reviews_df.loc[wine_reviews_df["Color_shades"] == "Unknown", "Color_shades"] = wine_reviews_df["Color"]


In [17]:
# Replace 'TEJO', 'TNTO' Color from Color column with Red,Rose, White, Yellow depends on color shades
# Define mapping of Color_Shades to actual Color values
shade_to_color_mapping = {
    'red': 'Red',
    'ruby': 'Red',
    'garnet': 'Red',
    'crimson': 'Red',
    'yellow': 'Yellow',
    'white': 'White',
    'rosé': 'Rosé',
    'pink': 'Rosé',
    'salmon': 'Rosé',
    'peach': 'Rosé',
    'coral': 'Rosé',
    'copper': 'Rosé',
    'gold' : 'Yellow',
    'amber': 'Yellow',
    'straw': 'White',
    'pale yellow': 'White',
    'light straw': 'White',
    'light amber': 'White',

}

# Replace 'TEJO' and 'TNTO' in Color column using Color_Shades
wine_reviews_df['Color'] =wine_reviews_df['Color_shades'].map(shade_to_color_mapping).fillna(wine_reviews_df['Color'])

print(wine_reviews_df)



                                       Name   Color  Alcohol_Percentage  \
0                                 .Beb 2007     Red                13.5   
1                                 .Beb 2009     Red                14.0   
2                                 .Beb 2010     Red                14.0   
3                                 .Beb 2011  Yellow                13.5   
4                                 .Beb 2012  Yellow                13.5   
...                                     ...     ...                 ...   
6343              Zé da Leonor Reserva 2013  Yellow                13.0   
6344                            Zéfyro 2007   White                13.0   
6345                            Zéfyro 2013  Yellow                12.5   
6346              Zom Touriga Nacional 2011     Red                14.5   
6347  Zonin delle Venezie Pinot Grigio 2010   White                12.0   

      Judge_Rating                                       Review_Notes  \
0             15.5  Bright

In [18]:
#replace the TNTO color with the red
wine_reviews_df.loc[wine_reviews_df['Color'] == 'TNTO', 'Color'] = 'Red'

In [19]:
wine_reviews_df['Color'].value_counts()

Color
Red       3698
White     1421
Yellow    1001
Rosé       228
Name: count, dtype: int64

In [20]:
wine_reviews_df.head()

,Name,Color,Alcohol_Percentage,Judge_Rating,Review_Notes,Wine_Bottle_Label,URL,Price,color,aromas,...,mouthfeel,aromas_aroma,categories,subcategories,outer_ring_attributes,Price_Interval,Alcohol_Content,Flavors Class,Color_shades,Sensation
0,.Beb 2007,Red,13.5,15.5,Bright red color and initial aromas dominated ...,It has an attractive color and an intense arom...,https://osvinhos.blogspot.com/2010/03/1232-beb...,5.75,"['red', 'Bright', 'rosé']","['initial aromas', 'red color', 'Bright red co...",...,[],"['persistent flavor', 'fruity notes', 'red col...",[],[],[],Low ($5 - $10),High (12% - 14%),Fruity,red,"fresh, delicate"
1,.Beb 2009,Red,14.0,16.0,"Reddish color and very aromatic nose, where th...",The careful choice of the best installments of...,https://osvinhos.blogspot.com/2013/08/2803-beb...,8.75,['red'],['aromatic nose'],...,"['seductive texture', 'round tannins']","['fresh set', 'seductive texture', 'subtle bar...",[],[],[],Low ($5 - $10),Very High (14%+),Fruity,red,"round, smooth, fresh, delicate, balanced"
2,.Beb 2010,Red,14.0,16.0,It presents a pleasantly concentrated ruby ​​c...,The careful choice of the best installments of...,https://osvinhos.blogspot.com/2014/01/2988-beb...,8.75,"['red', 'ruby']","['floral', 'fruity']",...,['round tannins'],"['Violet', 'delicate spicy notes', 'very fruit...",['Floral'],['Colored Flowers'],['Violet'],Low ($5 - $10),Very High (14%+),Fruity,"red, ruby","delicate, balanced, round"
3,.Beb 2011,Yellow,13.5,15.5,Bright yellow color and delicately marked nose...,We associated a very old wardrobe vineyard wit...,https://osvinhos.blogspot.com/2013/09/2820-beb...,8.75,"['Bright', 'white']",[],...,[],"['bright yellow color', 'floral touch', 'tropi...",[],[],[],Low ($5 - $10),High (12% - 14%),Floral,yellow,"fresh, balanced, smooth"
4,.Beb 2012,Yellow,13.5,16.0,It has a slightly pale yellow color and a nose...,We associated the structure and complexity of ...,https://osvinhos.blogspot.com/2015/06/3639-beb...,5.75,['white'],"['pale yellow color', 'yellow color', 'fruity'...",...,['well acidity'],"['well acidity', 'suggestive vegetable', 'flor...",['Maturation in Oak Barrel'],['Spices'],['Vanilla'],Low ($5 - $10),High (12% - 14%),Floral,yellow,"smooth, fresh, rich, balanced, elegant"


In [21]:
# check judge rating unique values
wine_reviews_df['Judge_Rating'].unique()

array([15.5, 16. , 15. , 17. , 16.5, 17.5, 14.5, 14. , 18.5, 13. , 13.5,
        0. , 12.5, 12. , 18. , 11.5, 19. ,  8. , 11. ])

In [22]:
# check if there is  Quinta Vale D. Maria 2016 in Name column
wine_reviews_df[wine_reviews_df['Name'].str.contains('Quinta Vale D. Maria 2016')]


,Name,Color,Alcohol_Percentage,Judge_Rating,Review_Notes,Wine_Bottle_Label,URL,Price,color,aromas,...,mouthfeel,aromas_aroma,categories,subcategories,outer_ring_attributes,Price_Interval,Alcohol_Content,Flavors Class,Color_shades,Sensation
5213,Quinta Vale D. Maria 2016,Red,15.0,17.0,Deep ruby ​​color and nose rich in fruity arom...,Bottled in August 2018.,https://osvinhos.blogspot.com/2020/11/5684-qui...,37.5,"['red', 'Deep', 'ruby']",['fruity'],...,['thin tannins'],"['black fruit', 'balsamic cut', 'toasted notes...",[],[],[],Mid-Range ($20 - $50),Very High (14%+),Fruity,"red, ruby","balanced, round, rich"


In [23]:
# combine Flavor and Color in one column 
wine_reviews_df["Flavor"] = wine_reviews_df["Color"] + " " + wine_reviews_df["Flavors Class"]

In [24]:
import pandas as pd
import re

# Define body classification dictionary
body_keywords = {
    "light_bodied": ["delicate", "crisp", "fresh", "ethereal", "light", "zesty", "vibrant", "airy", "lively", "subtle", "clean", "crunchy", "tart", "juicy", "brisk"],
    "medium_bodied": ["balanced", "smooth", "elegant", "moderate", "soft", "rounded", "supple", "silky", "mellow", "harmonious", "easygoing", "polished", "gentle", "fleshy", "lush", "creamy", "textured", "fleshy", "opulent", "generous", "plush", "voluptuous", "sensual", "fleshy", "opulent", "generous", "plush", "voluptuous", "sensual"],
    "full_bodied": ["rich", "bold", "velvety", "intense", "heavy", "lush", "powerful", "concentrated", "dense", "structured", "firm", "muscular", "robust", "weighty", "opulent", "generous", "plush", "voluptuous", "sensual", "fleshy", "opulent", "generous", "plush", "voluptuous", "sensual", "fleshy", "opulent", "generous", "plush", "voluptuous", "sensual"],
}

# Function to classify review notes
def classify_body(text):
    text = text.lower()  # Convert to lowercase
    for category, keywords in body_keywords.items():
        if any(re.search(rf'\b{kw}\b', text) for kw in keywords):
            return category  # Assign first matching category
    return "unknown"  # Default if no match

# Apply classification function
wine_reviews_df['Body_Type'] = wine_reviews_df['Review_Notes'].apply(classify_body)

# Display the results
wine_reviews_df.head()


,Name,Color,Alcohol_Percentage,Judge_Rating,Review_Notes,Wine_Bottle_Label,URL,Price,color,aromas,...,categories,subcategories,outer_ring_attributes,Price_Interval,Alcohol_Content,Flavors Class,Color_shades,Sensation,Flavor,Body_Type
0,.Beb 2007,Red,13.5,15.5,Bright red color and initial aromas dominated ...,It has an attractive color and an intense arom...,https://osvinhos.blogspot.com/2010/03/1232-beb...,5.75,"['red', 'Bright', 'rosé']","['initial aromas', 'red color', 'Bright red co...",...,[],[],[],Low ($5 - $10),High (12% - 14%),Fruity,red,"fresh, delicate",Red Fruity,light_bodied
1,.Beb 2009,Red,14.0,16.0,"Reddish color and very aromatic nose, where th...",The careful choice of the best installments of...,https://osvinhos.blogspot.com/2013/08/2803-beb...,8.75,['red'],['aromatic nose'],...,[],[],[],Low ($5 - $10),Very High (14%+),Fruity,red,"round, smooth, fresh, delicate, balanced",Red Fruity,light_bodied
2,.Beb 2010,Red,14.0,16.0,It presents a pleasantly concentrated ruby ​​c...,The careful choice of the best installments of...,https://osvinhos.blogspot.com/2014/01/2988-beb...,8.75,"['red', 'ruby']","['floral', 'fruity']",...,['Floral'],['Colored Flowers'],['Violet'],Low ($5 - $10),Very High (14%+),Fruity,"red, ruby","delicate, balanced, round",Red Fruity,light_bodied
3,.Beb 2011,Yellow,13.5,15.5,Bright yellow color and delicately marked nose...,We associated a very old wardrobe vineyard wit...,https://osvinhos.blogspot.com/2013/09/2820-beb...,8.75,"['Bright', 'white']",[],...,[],[],[],Low ($5 - $10),High (12% - 14%),Floral,yellow,"fresh, balanced, smooth",Yellow Floral,light_bodied
4,.Beb 2012,Yellow,13.5,16.0,It has a slightly pale yellow color and a nose...,We associated the structure and complexity of ...,https://osvinhos.blogspot.com/2015/06/3639-beb...,5.75,['white'],"['pale yellow color', 'yellow color', 'fruity'...",...,['Maturation in Oak Barrel'],['Spices'],['Vanilla'],Low ($5 - $10),High (12% - 14%),Floral,yellow,"smooth, fresh, rich, balanced, elegant",Yellow Floral,light_bodied


In [25]:
wine_reviews_df['Body_Type'].value_counts()

Body_Type
light_bodied     4237
medium_bodied    1242
unknown           484
full_bodied       385
Name: count, dtype: int64

In [29]:
wine_reviews_df.columns

Index(['Name', 'Color', 'Alcohol_Percentage', 'Judge_Rating', 'Review_Notes',
       'Wine_Bottle_Label', 'URL', 'Price', 'color', 'aromas', 'flavors',
       'finish', 'mouthfeel', 'aromas_aroma', 'categories', 'subcategories',
       'outer_ring_attributes', 'Price_Interval', 'Alcohol_Content',
       'Flavors Class', 'Color_shades', 'Sensation', 'Flavor', 'Body_Type'],
      dtype='object')

In [ ]:
# drop unnecessary columns
wine_reviews_df.drop(columns=['finish', 'mouthfeel', 'aromas_aroma', 'categories', 'subcategories',
       'outer_ring_attributes',], inplace=True)

In [27]:
# check if there is  Lemos Rosé 2020 in Name column
wine_reviews_df[wine_reviews_df['Name'].str.contains('Lemos Rosé 2020')]

,Name,Color,Alcohol_Percentage,Judge_Rating,Review_Notes,Wine_Bottle_Label,URL,Price,color,aromas,...,categories,subcategories,outer_ring_attributes,Price_Interval,Alcohol_Content,Flavors Class,Color_shades,Sensation,Flavor,Body_Type
2994,Lemos Rosé 2020,Rosé,9.0,15.5,"Very delicate salmon color, the nose suggests ...",Hugo Keys winemaker.,https://osvinhos.blogspot.com/2022/03/6180-lem...,8.75,['rosé'],"['floral', 'salmon color']",...,[],[],[],Low ($5 - $10),Low (0% - 10%),Floral,salmon,"fresh, delicate, dry",Rosé Floral,light_bodied


In [26]:
# save the cleaned data to a new csv file
wine_reviews_df.to_csv(r'C:\Users\fares\Final_project\processed_wine_data.csv', index=False)